### Imports

In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
import matplotlib.pyplot as plt

In [2]:
! pip install pyDOE2 #instalación de pyDOE2 local

In [3]:
import pyDOE2

In [4]:
#Se escribe p de planeamiento de los niveles del factor a emplear en las observaciones.

p = pyDOE2.fracfact('A B AB') # dimensión de la tabla para una fracción de resolución III
p = pd.DataFrame(p, columns=['A', 'B', 'C']) #factores 
p

,A,B,C
0,-1.0,-1.0,1.0
1,1.0,-1.0,-1.0
2,-1.0,1.0,-1.0
3,1.0,1.0,1.0


Como se observa cada fila se refiere a cada tratamiento a realizar y las columnas a los factores. El uno positivo y negativo son los niveles de cada factor correspondiente. Se debe recordar que la ejecución de cada experimento es de forma aleatoria, es decir, agarrar la columna izquierda que va de 0-3 y aleatorizar el orden.

### Adición de resultados a la tabla

In [5]:
#Al obtener los resultados en el laboratorio se computan en la tabla anterior como sigue:
p['resultados']=[73, 90, 19, 43]
p

,A,B,C,resultados
0,-1.0,-1.0,1.0,73
1,1.0,-1.0,-1.0,90
2,-1.0,1.0,-1.0,19
3,1.0,1.0,1.0,43


### Implementación de ANOVA en experimentos de fracciones factoriales


In [6]:
mod = ols('resultados~A+B+C', p).fit()
anova = sm.stats.anova_lm(mod)
anova

# Ocurre este error porque hay una cantidad muy pequeña de observaciones, sin embargo, 
# se puede analizar desde otra perspectiva

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/stats/anova.py:138: RuntimeWarning: divide by zero encountered in double_scalars
  (model.ssr / model.df_resid))


,df,sum_sq,mean_sq,F,PR(>F)
A,1.0,4.202500e+02,420.25,0.0,NaN
B,1.0,2.550250e+03,2550.25,0.0,NaN
C,1.0,1.225000e+01,12.25,0.0,NaN
Residual,0.0,2.524355e-28,inf,NaN,NaN


### ANOVA de alias 

#### Alias de A

In [7]:
mod = ols('resultados~B+C', p).fit()
anova = sm.stats.anova_lm(mod)
anova

,df,sum_sq,mean_sq,F,PR(>F)
B,1.0,2550.25,2550.25,6.068412,0.245491
C,1.0,12.25,12.25,0.029149,0.892347
Residual,1.0,420.25,420.25,NaN,NaN


#### Alias de B

In [8]:
mod = ols('resultados~A+C', p).fit()
anova = sm.stats.anova_lm(mod)
anova

,df,sum_sq,mean_sq,F,PR(>F)
A,1.0,420.25,420.25,0.164788,0.754509
C,1.0,12.25,12.25,0.004803,0.955948
Residual,1.0,2550.25,2550.25,NaN,NaN


#### Alias de C

In [9]:
mod = ols('resultados~A+B', p).fit()
anova = sm.stats.anova_lm(mod)
anova

,df,sum_sq,mean_sq,F,PR(>F)
A,1.0,420.25,420.25,34.306122,0.107653
B,1.0,2550.25,2550.25,208.183673,0.044052
Residual,1.0,12.25,12.25,NaN,NaN


Se ve que C es un factor que tiene cierto peso en en el los resultados porque un factor de su alias es estadisticamente significativo (menor a 0.05), pero no se puede concluir que sea un factor principal.